# Introduction to Data Science 
# Lecture 7: Statistical inference and hypothesis testing 
*COMP 5360 / MATH 4100, University of Utah, http://datasciencecourse.net/*

In this lecture, we'll cover 
* Statistical inference
* Central limit theorem
* Hypothesis testing and the z-test
* Confidence intervals
* A/B testing

Mandatory reading:
+ [WIRED article on A/B testing](http://www.wired.com/2012/04/ff_abtesting/)

Mandatory listening:
+ [Planet Money Episode 677: The experiment experiment](https://www.npr.org/sections/money/2018/03/07/591213302/episode-677-the-experiment-experiment)
+ [Planet Money Episode 669: A or B](https://www.npr.org/sections/money/2015/12/11/459412925/episode-669-a-or-b) 

Further reading: 
+ Jay L. Devore, Probability and Statistics for Engineering and the Sciences, 9th ed. Cengage Learning (2016) Ch. 8 and 9.
+ R. Nuzzo, Scientific method: Statistical errors, Nature (2014) [link](https://doi.org/10.1038/506150a)
+ J. Cohen, The Earth is Round (p<0.05), American Psychologist (1994) [link](https://doi.org/10.1037/0003-066x.49.12.997)


For a more complete treatment, take Math 3070 (Applied Statistics I).


In [ ]:
#imports and setup
import pandas as pd

import scipy as sc
from scipy.stats import bernoulli
from scipy.stats import binom
from scipy.stats import norm
from scipy.stats import t
from scipy.stats import probplot

import matplotlib.pyplot as plt
%matplotlib inline
plt.rcParams['figure.figsize'] = (10, 6)
plt.style.use('ggplot')

## Recap of Lecture 4: Descriptive vs. Inferential Statistics 

*Descriptive statistics* quantitatively describes or summarizes features of a dataset. 

*Inferential statistics* attempts to learn about the population from which the data was sampled.

## Recap of Lecture 4:  discrete random variables

*Discrete random variables* take discrete values with preassigned probabilities described by a probability mass function (PMF). If $X$ is the random variable and $f(k)$ is the PMF, we say "the probability that $X$ takes value $k$ is given by $f(k)$" and write
$$
\textrm{Prob}(X=k) = f(k).
$$

### Bernoulli distribution
A Bernoulli random variable can take the values $k=0$ or $1$ and has PMF
$$
f(k) = \begin{cases} p & k=1 \\ 1-p & k = 0 \end{cases}
$$


Some facts about Bernoulli variables: 
* mean is $p$
* variance is $p(1-p)$

**Example:** The Bernoulli distribution with $p=0.5$ describes a 'fair' coin toss where 1 and 0  represent "heads" and "tails", respectively. If the coin is unfair, then we would have that $p\neq 0.5$.

### Binomial distribution

A binomial r.v. takes values $k=0,1,\ldots,n$, with a probability given by the PMF 
$$
f(k) = \binom{n}{k} p^k (1-p)^{n-k}.
$$
Here, $\binom{n}{k} = \frac{n!}{k!(n-k)!}$ in the binomial coefficient that describes how many ways there are to choose a subset of $k$ elements, disregarding their order, from a set of $n$ elements.
 

In [ ]:
n =10
p = 0.5
f = lambda k: binom.pmf(k, n=n,p=p)

x = sc.arange(n+1);
plt.plot(x, f(x),'*-')
plt.title("The PMF for a binomial random variable")
plt.xlabel("k")
plt.ylabel("f(k)")
plt.xlim([0,n])
plt.show()

Some facts about the binomial distribution:
- A binomial random variable is just the sum of $n$ Bernoulli random variables. You can think of it as summarizing the resutls of $n$ coin flips by just keeping track of the total number of heads.
- The mean is  $np$
- The variance is  $np(1−p)$

### Poisson distribution
You also saw the Poisson random variable in the homework, which is another example of a discrete random variable. 

## Recap of Lecture 4:  continuous random variables

A *continuous random variable* can take any real value, but some numbers are more likely than others. The probability is given by the *probability density function (PDF)*, which is analogous to the PMF for discrete random variables. If f(x) is the PDF for the random variable $X$, then the probability that $X$ takes the value in the interval $[a,b]$ is given by 
$$
\textrm{Prob}(X\in[a,b]) = 
\int_a^b f(x) dx.
$$
This is just the area under the curve for this interval.

### Example: Normal (Gaussian) distribution 

The *probability density function (PDF)* for a normal (Gaussian) random variable is
$$
f(x) = \frac{1}{\sqrt{ 2 \pi \sigma^2 }}
e^{ - \frac{ (x - \mu)^2 } {2 \sigma^2} }.
$$
This is sometimes referred to as the 'bell curve'.

In [ ]:
mu = 0 # mean
sigma = 1 # standard deviation 
x = sc.arange(mu-4*sigma,mu+4*sigma,0.001);
pdf = norm.pdf(x,loc=mu, scale=sigma)
plt.title("The probability density function for a normal random variable")
plt.plot(x, pdf, linewidth=2, color='k')
plt.xlabel("x")
plt.ylabel("f(x)")
plt.show()

Some facts about the normal distribution:
- The mean is $\mu$
- The variance is  $\sigma^2$

To compute the integral 
$$
\textrm{Prob}(X\in[a,b]) = 
\int_a^b f(x) dx,
$$
it is useful to define the *cumulative distribution function* (CDF)
$$
F(x) = \int_{-\infty}^x f(x) dx.
$$
Then we can write 
$$
\int_a^b f(x) dx =
\int_{-\infty}^b f(x) dx  - \int_{-\infty}^a f(x) dx =
F(b) - F(a).
$$
This is convenient because we know longer have to evaluate an integral! However, there isn't a nice way to write $F(x)$ for the normal distribution in terms of elementary functions. So we just think about $F(x)$ as a known function that we can easily compute using python. 

In [ ]:
mu = 0 # mean
sigma = 1 # standard deviation 
x = sc.arange(mu-4*sigma,mu+4*sigma,0.001);
cdf = norm.cdf(x,loc=mu, scale=sigma)
plt.title("The cumulative density function for a normal random variable")
plt.plot(x, cdf, linewidth=2, color='k')
plt.xlabel("x")
plt.ylabel("F(x)")
plt.show()

### Exercise 
Interpret the following in terms of normal random variables:
- $\int_{-\infty}^1 f(x) dx = F(1)$

In [ ]:
norm.cdf(1, loc=mu, scale=sigma)

- $\int_{-1}^1 f(x) dx = F(1) - F(-1)$ 

In [ ]:
norm.cdf(1, loc=mu, scale=sigma) - norm.cdf(-1, loc=mu, scale=sigma) 

+ $\int_{-\infty}^\infty f(x) dx = 1$

In [ ]:
norm.cdf(sc.inf, loc=mu, scale=sigma)

+ $\int_1^1 f(x) dx = F(1) - F(1) = 0$


**Remark:** There are many other continuous random variables, but in this class we'll mostly only consider the normal random variable.

## Checking if a random variable is a normal random variable 

Given sample data, $x_1, x_2, x_3 \ldots$, how do you know if the data came from a normal distribution? 

+ There is a visual check called the "normal probability plot". 

In a [normal probability plot](https://en.wikipedia.org/wiki/Normal_probability_plot), the sorted data are plotted vs. values selected to make the points look close to a straight line if the data are approximately normally distributed. Deviations from a straight line suggest departures from normality. 

In [ ]:
samp_size = 200

x = norm.rvs(loc=0, scale=1, size=samp_size)
probplot(x, plot=plt)
plt.title("Normal probability plot")
plt.xlabel("Normal quantiles")
plt.ylabel("Ordered values")
plt.show()

x = t.rvs(df=3,size=samp_size)
probplot(x, plot=plt)
plt.title("Normal probability plot")
plt.xlabel("Normal quantiles")
plt.ylabel("Ordered values")
plt.show()

from scipy.stats import laplace
x = laplace.rvs(size=samp_size)
probplot(x, plot=plt)
plt.title("Normal probability plot")
plt.xlabel("Normal quantiles")
plt.ylabel("Ordered values")
plt.show()

from scipy.stats import gamma
x = gamma.rvs(a=2,size=samp_size)
probplot(x, plot=plt)
plt.title("Normal probability plot")
plt.xlabel("Normal quantiles")
plt.ylabel("Ordered values")
plt.show()

## Hypothesis testing

Suppose we have a coin and we want to determine whether or not it is 'fair'. We could flip it many, many times and count how many heads we obtain. If the fraction of heads is approximately $0.5$, we might argue that the coin is fair. 

This is an example of statistical inference. We are trying to determine something about the coin from samples of coin flips. 

Let's say we flip a coin $n=1000$ times. If the coin is fair, the outcome is described by the Binomial distribution with $p=0.5$. 

In [ ]:
f = lambda k: binom.pmf(k, n=1000,p=0.5)

x = sc.arange(1001);
plt.plot(x, f(x),'*-')
plt.plot(545,f(545),'o')
plt.title("The probability mass function for a Binomial random variable")
plt.xlabel("k")
plt.ylabel("f(k)")
plt.xlim([0,1001])
plt.show()

Suppose that in our experiment, we saw $545$ heads. The probability of this occurring is 
f(k = 545):

In [ ]:
binom.pmf(545, n=1000,p=0.5)

In hypothesis testing, the more important question is: what is the probability of seeing a value as extreme or more extreme than the value that we observed? 

We might say that any result $\geq 545$ is 'as or more extreme'.

So the probability of seeing as extreme of an outcome is:

In [ ]:
s = sum(binom.pmf(sc.arange(545,1001),n=1000,p=0.5))
print(s)
print(1-s)

Actually, seeing any result $\leq 455$ would be 'as or more extreme' too. Why?

So the probability of seeing as extreme of an outcome is:

In [ ]:
s1 = sum(binom.pmf(sc.arange(0,456),n=1000,p=0.5)) # why 456? 
s2 = sum(binom.pmf(sc.arange(545,1001),n=1000,p=0.5))
print(s1)
print(s2)
s = s1 + s2

print(s)
print(1-s)

So the probability of seeing so many heads or so many tails is just $0.49\%$. So it is very unlikely that if the coin were fair, we would see this result! Maybe so unlikely that we would declare that the coin is unfair? This is the idea behind **hypothesis testing**. 

**Note**: I didn't say that "it is unlikely that the coin is fair". But rather: "if the coin were fair, it would be unlikely to see this result". 

In *hypothesis testing*, we make a null hypothesis, written $H_0$. In this case, the null hypothesis is
$$
H_0: \text{the coin is fair, i.e., $p=0.5$}.
$$
The alternative hypothesis, $H_a$, is typically the hypothesis that the researcher wants to validate. In this case, $H_a$ is that the coin is unfair, i.e., $p\neq 0.5$. 

We also choose a *significance level* for the test, $\alpha$, traditionally $1\%$ or $5\%$. 
In this case, let's choose a significance level of $\alpha = 1\%$. 

We then perform an experiment. In this case, we flip the coin 1000 times and count the number of heads (in this case 545). 

Finally, assuming the null hypothesis is true, we compute how likely it is to see a number that is at least as far from the expected value as the number obtained. In our case, this is $0.49\%$. The is called the *p-value*. Since $p=0.49\%$ is smaller than the chosen significance level, $\alpha = 1\%$, we reject the null hypothesis and declare the coin to be unfair.  

Some comments about the p-value:
1. A p-value is a probability calculated assuming that $H_0$ is true. 
+ The smaller the p-value, the stronger the evidence against $H_0$.
+ **Warning:** A p-value is not the probability that the null hypothesis is true or false. It is the probability that an erroneous conclusion is reached assuming the null hypothesis to be true. In this example, it is the probability that the coin actually is fair and we just happened to see an outcome at least as extreme as 545 heads.

To avoid computing sums (as above) and to 'normalize' the above procedure, it is useful to introduce the *Central Limit Thoerem*. 

## Central Limit Theorem 

One of the reasons that the normal distribution is **so important** is the following theorem. 

**Central Limit Theorem.** Let $\{X_1,\ldots, X_n\}$ be a sample of $n$ random variables chosen identically and independently from a distribution with mean $\mu$ and finite variance $\sigma^2$. If $n$ is 'large', then 
- the sum of the variables $\sum_{i=1}^n X_i$ is also a random variable and is approximately **normally** distributed with mean $n\mu$ and variance $n\sigma^2$ and
- the mean of the variables $\frac{1}{n}\sum_{i=1}^n X_i$ is also a random variable and is approximately **normally** distributed with mean $\mu$ and variance $\frac{\sigma^2}{n}$.

How can we use the central limit theorem (CLT)? 

Recall that a binomial random variable is the sum of $n$ Bernoulli random variables. So the CLT tells us that if $n$ is large, binomial random variables will be distributed approximately normally. That is, if we flip a coin many times, the number of heads that we're likely to see is described by a normal distribution. This provides a different (easier) way to answer the question: How unlikely is it to flip a fair coin 1000 times and see 545 heads? 

Suppose we flip a fair ($p=0.5$) coin 1000 times. 

*Question:* How many heads do we expect to see? 

The CLT says that the number of heads (= sum of Bernoulli r.v. = binomial r.v.) is approximately normally distributed with mean 
$$ 
n\mu = np = 1000*0.5 = 500 
$$
and variance 
$$ 
n \sigma^2 = np(1-p) = 1000*0.5*0.5 = 250. 
$$

Let's do an experiment to see how good the CLT is for Bernoulli random variables. We'll call flipping a fair coin n=1,000 times and counting the number of heads a "simulation". Recall that the outcome is precisely a binomial random variable with n=1,000 and p = 0.5. We'll do 10,000 simulations and then compare the histogram of the binomial random variables and the normal distribution predicted by the CLT. 

In [ ]:
n = 1000
p = 0.5
bin_vars = binom.rvs(n=n,p=p,size=10000)

plt.hist(bin_vars, bins='auto',density=True)

mu = n*p 
sigma = sc.sqrt(n*p*(1-p))
x = sc.arange(mu-4*sigma,mu+4*sigma,0.1);
pdf = norm.pdf(x, loc=mu, scale=sigma)
plt.plot(x, pdf, linewidth=2, color='k')
plt.title("A comparison between the histogram of binomial random \n variables and the normal distribution predicted by the CLT")
plt.show()


probplot(bin_vars, plot=plt)
plt.title("Normal probability plot")
plt.xlabel("Normal quantiles")
plt.ylabel("Ordered values")
plt.show()


So what is the likelihood of flipping a coin 1000 times and seeing an event less extreme than 545 heads? 

The CLT tells us that this is approximately 
$$
\int_{455}^{545} f(x) dx = F(545) - F(455).
$$

This is something that we can easily evaluate using the cumulative distribution function (CDF). 

In [ ]:
n = 1000
p = 0.5
mu = n*p
sigma = sc.sqrt(n*p*(1-p))
print(norm.cdf(545, loc=mu, scale=sigma) - norm.cdf(455, loc=mu, scale=sigma)) 

# a plot illustrating the integral 
x = sc.arange(mu-4*sigma,mu+4*sigma,0.001);
plt.plot(x, norm.pdf(x, loc=mu, scale=sigma), linewidth=2, color='k')
x2 = sc.arange(455,545,0.001)
plt.fill_between(x2, y1= norm.pdf(x2,loc=mu, scale=sigma), facecolor='red', alpha=0.5)
plt.xlim([mu-4*sigma,mu+4*sigma])
plt.show()

So again, we see that $99.5\%$ of the time, we would see an event less extreme than 545 heads.  

## Example: "Freshman 15": Fact or Fiction

This example was taken from Devore, pp.314-315. 

"A common belief among the lay public is that body weight increases after entry into college, and the phrase 'freshman 15' has been coined to describe the 15 pounds that students presumably gain over their freshman year."

Let $\mu$ denote the true average weight gain in the first year of college. We take the null hypothesis to be
$$
H_0: \mu \geq 15
$$
so that the alternative hypothesis is that the average weight gain in the first year of college is less than 15 lbs ($H_a:  \mu < 15$). 

We set a significance level of, say, $\alpha = 1\%$. 

We suppose a random sample of $n$ students is selected, their weights (before and after the first year of college) are measured, and the sample mean $\bar{x}$ and sample standard deviation $s$ are computed. An article in the journal Obesity (2006) cites that for a sample of $n=137$ students, the sample mean weight gain was $\bar{x}=2.42$ lb and with a sample standard deviation of $s=5.72$ lb. 

Assuming $H_0$ to be true, how unlikely is it that we would observe such a small value ($\bar{x}=2.42$)?  

The CLT says that 
+ the mean of the variables $\frac{1}{n}\sum_{i=1}^n X_i$ is a random variable and is approximately **normally** distributed with mean $\mu$ and variance $\frac{\sigma^2}{n}$.


We take a normal distribution with mean given by the null value ($\mu = 15$) and variance given by $s^2/n = (5.72)^2/137=0.2388$. 

The $p$-value is then computed as the probability that $\bar{x}< 2.42$, 
$$
P(\bar{x}< 2.42) = \int_{-\infty}^{2.42} f(x) dx = F(2.42).
$$

In [ ]:
mu = 15
sigma = sc.sqrt(5.72**2/137)
print('p:', norm.cdf(2.42, loc=mu, scale=sigma))

The p-value is practically zero, much less than the significance level! The data very strongly contradicts the null hypothesis. We reject the null hypothesis, $H_0$, and conclude that the 'freshman 15' is fiction! 

## Example: Zinc in Batteries

This example was taken from Devore, pp.317. 

A random sample of $n=51$ Panasonic AAA batteries gave a sample mean zinc mass of $\bar{X}=2.06$ grams and a sample standard deviation of $S=0.141$ grams. 

**Question**: Does this data provide compelling evidence for concluding that the population mean zinc mass exceeds 2.0 grams, as reported by the company? 

Let $\mu$ denote the true average zinc mass of such batteries. We take the null hypothesis to be
$$
H_0: \mu \leq 2.0
$$
and that the alternative hypothesis is
$$
H_a:  \mu > 2.0.
$$

We set a significance level of, say, $\alpha = 1\%$. 

According to the CLT, the sample mean $\bar{X}$ has approximately a normal distribution with mean $\mu = 2.0$ (Assuming $H_0$) and standard deviation $S/\sqrt{n}$. We could proceed as previously but it is more standard to normalize the variable.  

To proceed, we conduct a **z-test**, which is the same as we did in the previous example, except now we use the normalized $Z$-statistic, 
$$
Z = \frac{\bar{X} - 2.0}{S/\sqrt{n}}. 
$$
The $Z$-statistic is distributed according to the "standard" normal distribution with mean $\mu=0$ and standard deviation $\sigma = 1$.

Assuming $H_0$ to be true, how unlikely is it that we would observe such a large value ($\bar{X}=2.06$)?
$$
P-\textrm{value} = P (\bar{X} \geq 2.06) = P (Z\geq 3.04) = \int_{3.04}^\infty f(x) dx = 1 - F(3.04). = .0012.  
$$
Because the $P$-value $= .0012 \leq .01 = \alpha$, the null hypothesis should be rejected at the chosen significance level. We conclude that the average zinc mass in the batteries exceeds 2.0. 

In [ ]:
print(1 - norm.cdf(2.06, loc=2.0, scale = 0.141/sc.sqrt(51)))

z = (2.06 - 2.0) / (0.141/sc.sqrt(51))
print(1 - norm.cdf(z, loc=0, scale=1))

## Summary of hypothesis testing and the z-test
1. Identify the parameter of interest and describe it in the context of the problem. 
+ Determine the null and alternative hypotheses.
+ Choose a significance level $\alpha$. 
+ Find the formula for the computed value of the test statistic, *e.g.*, 
$Z = \frac{\bar{X} - \mu}{\sigma/\sqrt{n}}$ (use the CLT).
+ Using the sampled data, compute the $P$-value, *e.g.*, $F(z)$
+ Compare the significance level to the $P$-value to decide whether or not the null hypothesis should be rejected and state the conclusion in the problem context. Report the $P$-value! 



### One- and two- sided hypothesis testing:

Depending on the null and alternative hypothesis, the $P$-value will be different integrals of the 'bell curve'. This is called [one- and two- sided hypothesis testing](https://en.wikipedia.org/wiki/One-_and_two-tailed_tests).

<img src="determinePvals.png" width="600">
$\qquad \qquad \qquad \qquad \qquad \qquad \qquad \qquad \qquad \qquad \qquad \qquad \qquad \qquad \qquad$ 
source: Devore, pp.329


## What to do for smaller sample sizes? Student's t-test

When $n$ is small, the Central Limit Theorem can no longer be used. In this case, if the samples are drawn from an approximately normal distribution, then the correct distribution to use is called the [Student's t distribution](https://en.wikipedia.org/wiki/Student%27s_t-distribution) with $\nu = n-1$ degrees of freedom. The probability density function (pdf) for the student's t distribution is not pretty (Google it!) but it is built into scipy, so we can compare the student's t-test to the normal distribution. 

In [ ]:
# there is some trouble with this package for some python versions
# if it doesn't work, don't worry about it
from ipywidgets import interact 

samp_mean = 0
samp_std_dev = 1

x = sc.linspace(samp_mean-4*samp_std_dev,samp_mean+4*samp_std_dev,1000);
def compare_distributions(sample_size):
    pdf1 = norm.pdf(x, loc=samp_mean, scale=samp_std_dev/sc.sqrt(sample_size))
    pdf2 = t.pdf(x,df=sample_size-1,loc=samp_mean, scale=samp_std_dev/sc.sqrt(sample_size))
    plt.plot(x, pdf1, linewidth=2, color='k',label='normal distribution pdf')
    plt.plot(x, pdf2, linewidth=2, color='r',label='t distribution pdf')
    plt.xlim(x.min(),x.max())
    plt.ylim(0,2)
    plt.legend()
    plt.show()
    
interact(compare_distributions,sample_size=(2,20,1))

The student's t distribution has "heavier tails" than the normal distribution. For a sample size greater than $\approx 20$, the normality assumption is generally accepted as reasonable. 

In the previous example, $n=51$, which is large enough to assume normality. 

## Types of error in hypothesis testing 

In hypothesis testing, there are [two types of errors](https://en.wikipedia.org/wiki/Type_I_and_type_II_errors). 
+ A **type I error** is the incorrect rejection of a true null hypothesis (a "false positive"). 
+ A **type II error** is incorrectly accepting a false null hypothesis (a "false negative"). 

Depending on the application, one type of error can be more consequential than the other. 

![](InferenceErrors.png)
$\qquad \qquad \qquad \qquad \qquad \qquad \qquad \qquad \qquad \qquad \qquad \qquad \qquad \qquad \qquad$ 
source: [wikipedia](https://en.wikipedia.org/wiki/Type_I_and_type_II_errors)

The probability of making a type I (false positive) error is the significance level $\alpha$. 

The probability of making a type II (false negative) error is more difficult to calculate. 

**Examples**

**(1)** In drug testing, we take the null hypothesis (H0): "This drug has no effect on the disease." A type I error detects an effect (the drug cures the disease) that is not present. A type II error fails to detect an effect (the drug cures the disease) that is present. 

**(2)** In a trial, we take the null hypothesis (H0): "This man is innocent." A type I error convicts an innocent person. A type II error lets a guilty person go free. 

## P hacking

Recall that the p-value measures how extreme the observation is and is compared to the significance level. Some comments about the p-value:
1. A p-value is a probability calculated assuming that $H_0$ is true. 
+ The smaller the p-value, the stronger the evidence against $H_0$.
+ A p-value is not the probability that the null hypothesis is true or false. It is the probability that an erroneous conclusion is reached.

Recently the *misuse* of hypothesis testing (p-values) has raised considerable controversy. Basically, if you do enough hypothesis tests, eventually you'll have a Type I (false positive) error. This is sometimes referred to as [Data dredging](https://en.wikipedia.org/wiki/Data_dredging). This is a real problem in a world with tons of data in which it is easy to do many, many hypothesis tests automatically. One method to avoid this is called *cross-validation*, which we'll discuss later. 

You can read more about 'P hacking' here:

- R. Nuzzo, Scientific method: Statistical errors, Nature (2014) [link](https://doi.org/10.1038/506150a)

- J. Cohen, The Earth is Round (p<0.05), American Psychologist (1994) [link](https://doi.org/10.1037/0003-066x.49.12.997)

- [Planet Money Episode 677: The experiment experiment](https://www.npr.org/sections/money/2018/03/07/591213302/episode-677-the-experiment-experiment)


## Confidence intervals

A [confidence interval](https://en.wikipedia.org/wiki/Confidence_interval) is an interval estimate for an unknown population parameter. For example, we might use collected data to give an interval estimate for a population mean. 

### Example: Ergonomic keyboards 

This example was taken from Devore, pp.277.

*Question*: A study is conducted to estimate the preferred height for an experimental keyboard with large forearm-wrist support. A sample of $n=31$ trained typists was selected, and the preferred keyboard height was determined for each typist. The resulting sample average preferred height was $\bar{X} = 80.0$ cm. Assuming that the preferred height is normally distributed with $\sigma = 2.0$ cm, obtain a confidence interval for $\mu$, the true average preferred height for the population of all typists. 

The CLT tells us that the sample mean $\bar X$ is normally distributed with expected value $\mu$ and standard deviation $\sigma / \sqrt n$. Standardizing $\bar X$ by subtracting the expected value and dividing by the standard deviation yields the standard normal variable 
$$
Z = \frac{\bar X - \mu}{\sigma / \sqrt n}.
$$

Since $Z$ is a standard normal variable, and the integral under the standard normal curve between -1.96 and 1.96 is 0.95, we have that 
$$
P( -1.96 < Z < 1.96) = 0.95.
$$

In [ ]:
norm.cdf(1.96) - norm.cdf(-1.96)

Let's manipulate the inequalities inside the parenthesis:
$$
-1.96 \ < \ \frac{\bar X - \mu}{\sigma / \sqrt n} \ < \ 1.96.
$$
This is equivalent to 
$$
\bar X -1.96 \frac{\sigma}{\sqrt n} \ < \ \mu \ < \ \bar X + 1.96\frac{\sigma}{\sqrt n}, 
$$
which we think of as the interval 
$$
I = \left( \bar X -1.96 \frac{\sigma}{\sqrt n}, \  \bar X +1.96 \frac{\sigma}{\sqrt n} \right) 
$$
containing the population mean $\mu$. 
The interval can be computed using the sampled data, $I = (79.3, 80.7)$. 
All together we have 
$$
P\left( I \textrm{ contains } \mu  \right) = 0.95.
$$
We say that $I = (79.3, 80.7)$ is the 95% confidence interval for the averaged preferred height. 

**Comments:**
+ The 95% confidence interval *does not* mean that with probability 95%, the true value of $\mu$ lies within the interval. 
+ A 95% confidence interval means that if we were to repeat the same experiment many times, and compute the confidence interval using the same formula, 95% of the time it would contain the true value of $\mu$.  

## A/B testing
*A/B testing* is a method of comparing two or more versions of an advertisement, webpage, app, etc. We set up an experiment where the variants are shown to users at random and statistical analysis is used to determine which is best. AB testing is the *de facto* test for many business decisions.  

**Example.** A/B testing was extensively used by President Obama during his 2008 and 2012 campaigns to  develop 
* optimized fund-raising strategies,  
* get-out-the-vote programs that would be most beneficial, and 
* target ads to the most susceptible audiences. 

Learn more here:
[Wired story on A/B testing](http://www.wired.com/2012/04/ff_abtesting/)
and 
[Planet Money Episode 669: A or B](https://www.npr.org/sections/money/2015/12/11/459412925/episode-669-a-or-b)

**Example.** Suppose your company is developing an advertisement. The art department develops two internet ads: "Ad A" and "Ad B". Your job is to figure out which is better. 

You decide to do an experiment: You use Google ads to randomly show 1000 internet users Ad A and 1000 internet users Ad B. 

It turns out that 500 Ad A viewers click on the ad while 550 Ad B viewers click on the ad? Obviously Ad B did better, but is the difference "significant" enough to say that Ad B is better? Or perhaps Ad B just got lucky in this test? 

In homework 4, you’ll answer this question. More generally, this is a question about the difference between population proportions. 

## Statistical inference for a difference between population proportions
We consider comparing the population proportions of two different populations. 

We make the following definitions:
- $N_A$ is the number of surveyed people from population $A$  
- $n_A$ is the number of successes from population $A$
- $p_A = n_A/N_A$ is the proportion of successes from population $A$

Similarly, we define 
- $N_B$ is the number of surveyed people from population $B$  
- $n_B$ is the number of successes from population $B$
- $p_B = n_B/N_B$ is the proportion of successes from population $B$

We make the null hypothesis:
$$
H_0\colon \text{$p_A$ and $p_B$ are the same, that is, } p_A - p_B = 0.
$$
That is, the proportion of successes in the two populations is the same. 

We'll take it as a fact (see Devore Ch. 9.4 or Math 3070) that: 
- $n_A/N_A$ is approximately a normal random variable with mean $p_A$ and variance $\sigma_A^2 = p_A(1-p_A)/N_A$ 
- $n_B/N_B$ is approximately a normal random variable with mean $p_B$ and variance $\sigma_B^2 = p_B(1-p_B)/N_B$
- $n_A/N_A - n_B/N_B$ is approximately a normal random variable with mean $\mu = 0$ and variance $\sigma^2 = \sigma_A^2 + \sigma_B^2$. 
- The test statistic called the *two-proportion z-value* 
$$
Z = \frac{p_A - p_B}{\sqrt{\hat{p} \hat{q} \left( \frac{1}{N_A} + \frac{1}{N_B} \right)}}.
$$
is approximately  distributed according to the standard normal distribution when $H_0$ is true. Here $\hat{p} = \frac{N_A}{N_A + N_B}p_A + \frac{N_B}{N_A + N_B}p_B$ and $\hat{q} = 1-\hat{p}$. 

From the data, we estimate the mean, $\mu$, to be  $p_A - p_B$. 

## Example: 1954 Salk polio-vaccine experiment

In 1954, polio was widespread and a new vaccine of unknown efficacy was introduced. To test the efficacy, in a double-blind study, two groups of children were give injections: one contained the vaccine and the other contained a placebo. 

Let $p_A$ and $p_B$ be the proportions of the children, having received the placebo and vaccine injections, respectively, to contract polio. We formulate the null hypothesis that 
$$
H_0\colon p_A - p_B \leq 0,
$$
that is, the vaccine is not effective.
The alternative hypothesis is that 
$$
H_a\colon p_A - p_B >0, 
$$
that is, a vaccinated child is less likely to contract polio than a child receiving the placebo.

We choose a significance level of $\alpha = 0.01$. 

An experiment was conducted with the following results: 
$$
\begin{aligned}
&\text{Placebo:} \quad N_A = 201,229, \quad n_A = 110 \\
&\text{Vaccine:} \quad N_B = 200,745, \quad n_B = 33.
\end{aligned}
$$

In [ ]:
nA = 110
NA = 201229
pA = nA/NA
muA = pA
sigmaA = sc.sqrt(pA*(1-pA)/NA)

nB = 33
NB = 200745
pB = nB/NB
muB = pB
sigmaB = sc.sqrt(pB*(1-pB)/NB)

Now we perform the hypothesis test and see what the probability of the outcome is under the assumption of the null hypothesis. 

In [ ]:
phat = NA*pA/(NA+NB) + NB*pB/(NA+NB)
qhat = 1-phat

z = (pA - pB)/sc.sqrt(phat*qhat*(1/NA + 1/NB)) 
print(z)

p_value = 1-norm.cdf(z)
print(p_value)

The probability that an erroneous conclusion is reached, under the assumption of the null hypothesis, is $6.6\times10^{-11}$, way less than the significance level, $\alpha$. We reject the null hypothesis and declare that the vaccine is more effective than a placebo! 